## Config Cell
## [Jump to outputs](#Outputs)

In [1]:
# Add shapefile
shp = "input/Lake Chad.geojson"
# shp = "input/CMR_ADM0_20KM-B_3857.geojson"
buffer = 0  # boundary buffer
cell_size = 0.1  # grid cell size

# Run 1. aoi-threshold.ipynb to get the value of th_aoi and store it here.
th_aoi = -21
# Set test_cells value if you need to test with initial 'n' cells instead of the entire grid
test_cells = 2
# Set run_checks to True if you want to run checks
run_checks = False

# Create folder ids by copying the ID from the g-drive folder url. If not using a particular ID, set as None
F_MN_FID = "1Tq_piYp8TCNBf3-qaEoTy6IZ_OreppvL"
F_MD_FID = None
PRF_MN_FID = None
PRF_MD_FID = None
POF_MN_FID = None
POF_MD_FID = None
FE_MN_FID = None
FE_MD_FID = None
MF_MX_FID = "1rLnPb_hrXdve-mfy5wSjsU2TNd7HfZQu"

cwc_name = "CMR_CWC_1123-1124"

# Define main time period of analysis
timerange = ("2023-11", "2024-11")
# Define sub-periods of analysis - should be within main time period
pre_flood = []
flood = ["2023-11", "2024-11"]
post_flood = []
max_flood = []
# Analysis periods
a_periods = ["flood"]
# Required Analysis measurements
a_measures = ["mean"]  # a_measures = ["mean", "median"]

## Init Requirements

## Dependencies

In [2]:
%matplotlib inline
# !pip install python-dotenv
# load_dotenv()
# from dotenv import load_dotenv

import os, glob, warnings, datacube, rasterio, folium, json
import numpy as np
import xarray as xr
import geopandas as gpd
import matplotlib.pyplot as plt
import rioxarray as rio
import pandas as pd
from rasterio.merge import merge
from rasterio.plot import show
from shapely.geometry import Point
from shapely.geometry import Polygon
from datetime import date
from dateutil.relativedelta import relativedelta


from scipy.ndimage import uniform_filter
from scipy.ndimage import variance
from skimage.filters import threshold_minimum
from datacube.utils.geometry import Geometry

from deafrica_tools.spatial import xr_rasterize
from deafrica_tools.datahandling import load_ard
from deafrica_tools.plotting import display_map, rgb
from deafrica_tools.areaofinterest import define_area

from typing import Literal

from IPython.display import clear_output
from IPython.display import display

warnings.filterwarnings("ignore")

In [3]:
# Init Google Drive (GCS) module
from tools.gdrive import GDrive

gd = GDrive()

In [4]:
gd.get_storage()

Google Drive Storage


,Loc,Size in GB
0,Capacity,2199
1,Available,1837
2,Usage,362
3,Drive Usage,362
4,Trash Usage,0


## Connect to the datacube

Connect to the datacube so we can access DEA data.
The `app` parameter is a unique name for the analysis which is based on the notebook file name.

In [5]:
dc = datacube.Datacube(app="Radar_water_detection")

## Filter and Classifier Functions

In [6]:
# Function to apply lee filtering on S1 image. Speckle Filter
def lee_filter(da, size):
    """
    Apply lee filter of specified window size.
    Adapted from https://stackoverflow.com/questions/39785970/speckle-lee-filter-in-python

    """
    img = da.values
    img_mean = uniform_filter(img, size)
    img_sqr_mean = uniform_filter(img**2, size)
    img_variance = img_sqr_mean - img_mean**2

    overall_variance = variance(img)

    img_weights = img_variance / (img_variance + overall_variance)
    img_output = img_mean + img_weights * (img - img_mean)

    return img_output


# Classifier Function
def S1_water_classifier(da, threshold):
    water_data_array = da < threshold
    return water_data_array.to_dataset(name="s1_water")

## Operational Funtions

In [7]:
def get_months(timerange: tuple) -> list[date]:
    """
    Converts the timerange into a list of months.

    Parameters:
    timerange ("YYYY-MM", "YYYY-MM"): tuple, required
        Timerange of the analysis. Start month to end month.

    Returns:
    months: list[date]
    """
    syear, smonth = timerange[0].split("-")
    syear = int(syear)
    smonth = int(smonth)
    start = date(syear, smonth, 30)

    eyear, emonth = timerange[1].split("-")
    eyear = int(eyear)
    emonth = int(emonth)
    end = date(eyear, emonth, 30)
    months = []
    while start <= end:
        months.append(start.strftime("%Y-%m"))
        start = start + relativedelta(months=+1)

    return months

In [8]:
# Generate raster outputs
def gen_output(
    DS: xr.DataArray,
    i: list,
    poly: Geometry,
    cell: int,
    aoi_m: list,
    period: Literal["preflood", "flood", "postflood", "flood_extents", "maxflood"],
    measure: Literal["mean", "median", "max"],
) -> list:
    """
    Generates file names and uploads tp Google Drive

    Parameters:
    DS: xr.DataArray, required
        Dataset to be converted into raster output.
    i: list, required
        Index of centroid list.
    poly: Geometry, required
        Polygon feature used to create the DS.
    cell: int, required
        Cell number being processed.
    aoi_m: list, required
        List of geojson feature collection (cells) that make up the entire grid.
    period: Literal["preflood", "flood", "postflood", "flood_extents"], required
        Time period of the process - PRE_FLOOD, FLOOD.
    measure: Literal["mean", "median"], required
        Central tendency measurement of the DS - mean, median.

    Returns:
    err: list
        Error log list of [x, y, cell_id and None]. None will store the error "P" - Processing or "U" - Upload.
    """

    err = []

    pm_dict = {
        "preflood_mean": [pre_flood, PRF_MN_FID],
        "preflood_median": [pre_flood, PRF_MD_FID],
        "flood_mean": [flood, F_MN_FID],
        "flood_median": [flood, F_MD_FID],
        "postflood_mean": [post_flood, POF_MN_FID],
        "postflood_median": [post_flood, POF_MD_FID],
        "flood_extents_mean": [timerange, FE_MN_FID],
        "flood_extents_median": [timerange, FE_MD_FID],
        "maxflood_max": [timerange, MF_MX_FID],
    }

    poly_gdf = gpd.GeoDataFrame(geometry=[poly], crs=poly.crs)
    lat_range = (poly_gdf.total_bounds[1], poly_gdf.total_bounds[3])
    lon_range = (poly_gdf.total_bounds[0], poly_gdf.total_bounds[2])
    g = poly_gdf.centroid

    data_val = "CELL_" + str(i[2]) + "_{}_{}".format(period.upper(), measure.upper())
    data_name = data_val + ".tif"
    data_out = "output/{}/".format(period) + data_name
    DS.rio.to_raster(data_out)

    # preflood meta
    data_dict = {
        "GRID_CELL_ID": i[2],
        "start_time": pm_dict["{}_{}".format(period, measure)][0][0],  # pre_flood[0]
        "end_time": pm_dict["{}_{}".format(period, measure)][0][-1],  # pre_flood[-1]
        "lat": lat_range,
        "lon": lon_range,
        "centroid": "{}, {}".format(g.y[0], g.x[0]),
        "crs": str(poly.crs),
    }

    text_flie_name = data_val + "_META.json"
    data_meta_path = "output/{}/".format(period) + text_flie_name
    with open(data_meta_path, "w") as f:
        json.dump(data_dict, f)

    try:
        gd.upload_files(
            [data_out, data_meta_path], pm_dict["{}_{}".format(period, measure)][1]
        )  # pm_dict['preflood_mean'][1] = FOLDER_ID
    except Exception as e:
        err.append([g.x[0], g.y[0], i[2], "U-PRF"])
        print(
            "\n\n"
            + "\033[31m"
            + "ERROR UPLOADING GRID CELL ID {} NO.  {}/{} CENTROID ({}, {}). LOGGED CENTROID INFO in e_log".format(
                i[2], cell, len(aoi_m), round(g.y[0], 5), round(g.x[0], 5)
            )
            + "\033[0m"
        )
        print("UPLOAD ERROR: {}".format(e))

    return err

In [9]:
def gen_elog(e_log: list) -> list:
    """
    Writes a the error log json file and uploads it to the google drive folder ID, if specified.

    Parameters:
    e_log: list, required
        Error log list of [x, y, cell_id and None]. None will store the error "P" - Processing or "U" - Upload.

    Returns:
    e_log: list
        Error log list having the same values as the input parameter
    """
    e_log = np.array(e_log)
    with open("error_centroids.json", "w") as filehandle:
        json.dump(e_log.tolist(), filehandle)

    # read error log from disk
    with open("error_centroids.json") as f:
        e_log = json.load(f)
    for e in e_log:
        e[0] = float(e[0])
        e[1] = float(e[1])
        e[2] = int(e[2])

    try:
        gd.upload_files(["error_centroids.json"], ERR_FOLDER_ID, False)
    except Exception as e:
        print("FAILED TO UPLOAD ERROR LOG FILE REASON:{}".format(e))

    return e_log

In [10]:
def gen_table():
    # Init pd.DataFrame
    headers = ["cell_id", "total", "lat", "lon", "datasets"]
    months = get_months(timerange)
    headers.extend(months)
    count_table = pd.DataFrame(columns=headers)
    return count_table

In [11]:
# Iterate through the input grid
def iterate_grid(aoi_m: list, c: list, count_table) -> list:
    """
    Iterates through every feature (cell) in the AOI grid.

    Parameters:
    aoi_m: list, required
        List of geojson feature collection (cells) that make up the entire grid.
    c: list, required
        List of [x, y, cell_id and None]. None will store the error "P" - Processing or "U" - Upload, if executio fails
    count_table: pd.DataFrame, optional
        Water pixel count table.

    Returns:
    e_log: list
        Error log list of [x, y, cell_id and None]. None will store the error "P" - Processing or "U" - Upload.
    """

    e_log = []
    cell = 1
    months = months = get_months(timerange)

    # Run the main iterator
    for aoi, i in zip(aoi_m, c):
        geopolygon = Geometry(aoi["features"][0]["geometry"], crs="epsg:4326")
        geopolygon_gdf = gpd.GeoDataFrame(geometry=[geopolygon], crs=geopolygon.crs)
        g = geopolygon_gdf.centroid
        print(
            "\n\n"
            + "\033[32m"
            + "PROCESSING GRID CELL ID {} NO. {}/{} CENTROID ({}, {})".format(
                i[2], cell, len(aoi_m), round(g.y[0], 5), round(g.x[0], 5)
            )
            + "\033[0m"
        )

        # Get the latitude and longitude range of the geopolygon
        lat_range = (geopolygon_gdf.total_bounds[1], geopolygon_gdf.total_bounds[3])
        lon_range = (geopolygon_gdf.total_bounds[0], geopolygon_gdf.total_bounds[2])

        # Load Sentinel1 data
        try:
            S1 = load_ard(
                dc=dc,
                products=["s1_rtc"],
                # measurements=["vv", "vh"],
                measurements=["vh"],
                y=lat_range,
                x=lon_range,
                time=timerange,
                output_crs="EPSG:6933",
                resolution=(-20, 20),
                group_by="solar_day",
                dtype="native",
            )
        except Exception as e:
            # Log error aoi centroids and keep looping
            e_log.append([g.x[0], g.y[0], i[2], "P"])
            print(
                "\n\n"
                + "\033[31m"
                + "ERROR PROCESSING GRID CELL {}/{} CENTROID ({}, {}). LOGGED CENTROID INFO in e_log".format(
                    i[2], len(aoi_m), round(g.y[0], 5), round(g.x[0], 5)
                )
                + "\033[0m"
            )
            print("PROCESS ERROR: {}".format(e))
            cell += 1
            continue

        datasets = S1.time
        dn = len(datasets.time)

        # Initialize row for the count_table
        count_row = [i[2], len(aoi_m), round(g.y[0], 5), round(g.x[0], 5), dn]

        # timesteps = [2, 4, 6, 9, 11]

        # The lee filter above doesn't handle null values
        # We therefore set null values to 0 before applying the filter
        valid = np.isfinite(S1)
        S1 = S1.where(valid, 0)

        # Create a new entry in dataset corresponding to filtered VV and VH data
        S1["filtered_vh"] = S1.vh.groupby("time").apply(lee_filter, size=7)

        # Null pixels should remain null
        S1["filtered_vh"] = S1.filtered_vh.where(valid.vh)

        # Convert the digital numbers to dB
        S1["filtered_vh"] = 10 * np.log10(S1.filtered_vh)

        threshold_vh = th_aoi

        S1["water"] = S1_water_classifier(S1.filtered_vh, threshold_vh).s1_water
        S1Water = S1.water
        S1_BIN = S1Water.where(S1Water > 0)
        FS1 = S1_BIN
        PRFS1 = S1_BIN

        # Generate CSV
        print("Generating row data...")
        for month in months:
            S1_month = S1_BIN.sel(time=[month], method="nearest").mean(dim="time")
            # Get count of all pixels that are water and not np.nan
            count = np.count_nonzero(~np.isnan(S1_month.values))
            count_row.append(count)
        print(count_row)
        print("Adding to table")
        count_table = pd.concat(
            [pd.DataFrame([count_row], columns=count_table.columns), count_table],
            ignore_index=True,
        )
        count_table.to_csv("output/csv/{}.csv".format(cwc_name), index=False, mode="w")

        # Creating outputs
        # Export to raster - upload to g-drive - delete from sandbox
        print("Analysis periods:")
        for p in a_periods:
            print(p)

        print("\nUploading...")
        # -------------- maxflood ----------------
        if "max_flood" in a_periods:
            if i[3] in [None, "P", "U-PRF"]:
                S1_MX = S1_BIN.sel(time=max_flood, method="nearest").max(dim="time")
                err = gen_output(S1_MX, i, geopolygon, cell, aoi_m, "maxflood", "max")

        # # -------------- preflood ----------------
        if "pre_flood" in a_periods:
            if i[3] in [None, "P", "U-PRF"]:
                S1_PRF_MD = PRFS1.sel(time=pre_flood, method="nearest").median(
                    dim="time"
                )
                S1_PRF_MN = PRFS1.sel(time=pre_flood, method="nearest").mean(dim="time")
                if "median" in a_measures:
                    err = gen_output(
                        S1_PRF_MD, i, geopolygon, cell, aoi_m, "preflood", "median"
                    )
                    if len(err) > 0:
                        e_log.extend(err)
                if "mean" in a_measures:
                    err = gen_output(
                        S1_PRF_MN, i, geopolygon, cell, aoi_m, "preflood", "mean"
                    )
                    if len(err) > 0:
                        e_log.extend(err)

        # # --------------- flood ------------------
        if "flood" in a_periods:
            if i[3] in [None, "P", "U-F"]:
                S1_F_MD = FS1.sel(time=flood, method="nearest").median(dim="time")
                S1_F_MN = FS1.sel(time=flood, method="nearest").mean(dim="time")
                if "median" in a_measures:
                    err = gen_output(
                        S1_F_MD, i, geopolygon, cell, aoi_m, "flood", "median"
                    )
                    if len(err) > 0:
                        e_log.extend(err)
                if "mean" in a_measures:
                    err = gen_output(
                        S1_F_MN, i, geopolygon, cell, aoi_m, "flood", "mean"
                    )
                    if len(err) > 0:
                        e_log.extend(err)

        # # ------------ flood-extents --------------
        if "flood_extents" in a_periods:
            if i[3] in [None, "P", "U-FE"]:
                S1_FE_MD = S1_F_MD - S1_PRF_MD
                S1_FE_MN = S1_F_MN - S1_PRF_MN
                if "median" in a_measures:
                    err = gen_output(
                        S1_FE_MD, i, geopolygon, cell, aoi_m, "flood_extents", "median"
                    )
                    if len(err) > 0:
                        e_log.extend(err)
                if "mean" in a_measures:
                    err = gen_output(
                        S1_FE_MN, i, geopolygon, cell, aoi_m, "flood_extents", "mean"
                    )
                    if len(err) > 0:
                        e_log.extend(err)

        cell += 1
        clear_output()

    if len(e_log) == 0:
        print(
            "\n\n"
            + "\033[32m"
            + "GRID PROCESSED AND UPLOADED SUCCESSFULLY"
            + "\033[0m"
            + "\n\n"
        )
    else:
        e_log = gen_elog(e_log)

    # return e_log to be run again
    return e_log

In [12]:
# Crete the aoi-mosaic - aoi_m
def gen_aoim(c: list, b: float, count_table) -> list:
    """
    Generates the feature collection list (list of cells) using centroid coordinates and a buffer distance. Calls the main iterator for execution as well.

    Parameters:
    c: list, required
        List of [x, y, cell_id and None]. None will store the error "P" - Processing or "U" - Upload, if executio fails.
    b: float, required
        Cell half-dimension in degrees (EPSG:4326). Creates a cell by adding this distance to the centroid coordinates.
    count_table: pd.DataFrame, optional
        Water pixel count table.

    Returns:
    e_log: list
        Error log list of [x, y, cell_id and None]. None will store the error "P" - Processing or "U" - Upload.
    """
    aoi_m = []
    for i in c:
        aoi_m.append(define_area(i[1], i[0], buffer=b))
    # print(c, len(aoi_m))
    e_log = iterate_grid(aoi_m, c, count_table)

    # return e_log to be run again
    return e_log

In [13]:
# Visualize input file
def view_input(gdf_list: list[gpd.GeoDataFrame], grid_c: list) -> None:
    """
    Visualizes cells and respective IDs  on a basemap.

    Parameters:
    gdf_list:list[gpd.GeoDataFrame], required
        List of geodataframes to be visualized.
    grid_c:list, required
        List of [x, y, cell_id and None]. None will store the error "P" - Processing or "U" - Upload, if executio fails.

    Returns:
    None

    """
    print("Visualizing data...")
    p = gdf_list[0].dissolve()
    center = p.centroid
    map = folium.Map(location=[center.y, center.x], tiles="CartoDB Positron")

    for gdf in gdf_list:
        folium.GeoJson(gdf, name="{}".format(gdf)).add_to(map)

    for c in grid_c:
        folium.Marker(
            location=[c[1], c[0]],
            popup=f"Centroid: {c[1]}, {c[0]}",
            icon=folium.DivIcon(
                icon_size=(10, 10),
                icon_anchor=(0, 0),
                html='<div style="font-size: 10pt">{}</div>'.format(c[2]),
            ),
        ).add_to(map)

    bounds = gdf_list[0].total_bounds.tolist()
    map.fit_bounds([bounds[:2][::-1], bounds[2:][::-1]])
    display(map)

In [14]:
# Create grid
def create_grid(adm0: gpd.GeoDataFrame, size: float) -> gpd.GeoDataFrame:
    """
    Divides adm0 AOI vectorfile into square grid based on size

    Parameters:
    adm0:gpd.GeoDataFrame, required
        AMD0 GeoDataFrame created from ADM0 input vector file
    size:float, required
        Grid cell size in degrees (EPSG:4326)

    Returns:
    grid: gpd.GeoDataFrame
        The generated grid GeoDataFrame
    """
    bounds = adm0.bounds
    minx = bounds.minx[0]  # only 1 feature at the 0th index
    miny = bounds.miny[0]
    maxx = bounds.maxx[0]
    maxy = bounds.maxy[0]

    grid = gpd.GeoDataFrame()
    for x0 in np.arange(minx, maxx, size):
        for y0 in np.arange(miny, maxy, size):
            x1 = x0 + size
            y1 = y0 + size
            d = {"geometry": [Polygon([(x0, y0), (x1, y0), (x1, y1), (x0, y1)])]}
            cell = gpd.GeoDataFrame(d, crs="EPSG:4326")
            flag = adm0.intersection(cell)
            if flag[0].is_empty == False:
                grid = pd.concat([grid, cell])

    return grid

In [15]:
# Check CRS and convert to 4326 if required
def crs_check(shp: gpd.GeoDataFrame) -> gpd.GeoDataFrame:
    """
    Checks input GeoDataFrame CRS and converts to EPSG 4326, if different.

    Parameters:
    shp: gpd.GeoDataFrame, required
        Input GeoDataFrame to check.

    Returns:
    shp: gpd.GeoDataFrame
        As is or converted GeoDataFrame.
    """
    if shp.crs != "EPSG:4326":
        print("Added ADM0 CRS is {}. Converting to EPSG:4326...".format(shp.crs))
        shp = shp.to_crs("EPSG:4326")
        if shp.crs == "EPSG:4326":
            print("Done")

    return shp

In [16]:
def check_inp(x: str) -> None:
    """
    Checks if input is "y" or "n"

    Parameters:
    x: str, required
        String input to be checked

    Returns:
    None
    """
    if x not in ["y", "n"]:
        raise ValueError("Invalid input, must be 'y' or 'n'")
    elif x == "n":
        raise RuntimeError(
            "Excecution terminated. Make necessary changes before running again"
        )


def exec_checks(c, buffer, count_table) -> list:
    """
    Performs checks and Run the entire application

    Parameters:
    c: list, required
        List of [x, y, cell_id and None]. None will store the error "P" - Processing or "U" - Upload, if executio fails.
    buffer: float, required
        Cell half-dimension in degrees (EPSG:4326). Creates a cell by adding this distance to the centroid coordinates.

    Returns:
    Returns:
    e_log: list
        Error log list of [x, y, cell_id and None]. None will store the error "P" - Processing or "U" - Upload.
    """

    inst = """
Before running the execution, ensure all requirements have been met:
1. Create appropriate folders in Google Drive and add their Folder IDs here
2. Check input shapefile
3. Check grid size
    
    """
    print(inst)

    x = input("Folder IDs verified? (y/n):")
    check_inp(x)

    x = input("Input shapefile/geojson verified? (y/n):")
    check_inp(x)

    input("Grid (size) verified? (y/n):")
    check_inp(x)

    z = input("\nBegin execution for entire input shapefile/geojson? (y/n):")
    if z not in ["y", "n"]:
        raise ValueError("Invalid input, must be 'y' or 'n'")
    elif z == "n":
        raise RuntimeError(
            "Excecution terminated. Make necessary changes before running again"
        )
    elif z == "y":
        print("Starting execution...")
        # get e_log with centroids, cell_id and error message
        # Calling gen_aoim will run the entire iteration process
        e_log = gen_aoim(c, buffer, count_table)
        print(len(e_log))
        return e_log

In [17]:
def del_files(path: str, ext: str) -> None:
    """
    Deletes all files with specified extention at specified folder path

    Parameters:
    path:str, required
        Folder path.
    ext:str, required
        File extension. "*" for all files.

    Returns:
    None
    """
    res_files = False
    loc = os.path.join(path, ext)
    files = glob.glob(loc)
    if len(files) > 0:
        res_files = True
        print("Found {} files. Deleting...".format(len(files)))
        for f in files:
            os.remove(f)

    return res_files

In [18]:
def clean_dirs() -> None:
    """
    Creates directories if they dont exist or deletes residual files if they exist.

    Parameters:
    None

    Returns:
    None
    """
    dirs_exist = False
    dir_dict = {
        "sd_flood": "output/flood",
        "sd_preflood": "output/preflood",
        "sd_flood_extents": "output/flood_extents",
    }

    for k in dir_dict:
        if not os.path.exists(dir_dict[k]):
            os.makedirs(dir_dict[k])
        else:
            dirs_exist = True
            r = del_files(dir_dict[k], "*")

    if dirs_exist:
        print("Output folders alredy exist.")
    else:
        print("Output folders created.")

    if not r:
        print("No residual files to delete.")

In [19]:
def create_shapefile(centroids, filename, preview=False):
    # Export grid polygon
    o_grid = gpd.GeoDataFrame()
    for i in centroids:
        point = Point(i[0], i[1])  # This takes x first and then y
        gdf = gpd.GeoDataFrame(geometry=[point])
        buffer = cell_size / 2
        cell = gpd.GeoDataFrame()
        cell["geometry"] = gdf.buffer(buffer, cap_style="square")
        cell.set_geometry("geometry")
        cell["cell_id"] = i[2]
        o_grid = pd.concat([o_grid, cell])

    o_grid = o_grid.set_crs("epsg:4326")
    o_grid.to_file("output/shape/{}_grid.geojson".format(filename), driver="GeoJSON")
    if preview:
        view_input([o_grid], centroids)

In [20]:
def add_adm(
    shp: str, boundary_buffer: float, cell_size: float, test_cells: int = None
) -> list | gpd.GeoDataFrame | pd.DataFrame:
    """
    Processes the input ADM file. File does not have to be ADM0 and can be any vectorfile.

    Parameters:
    shp: str, required
        Path of input vectorfile. Will be converted to EPSG:4326.
    boundary_buffer: float, required
        Outer boundary buffer to be given to the input vector file in degrees (EPSG:4326)
    cell_size: float, required
        Grid cell size in degrees (EPSG:4326)

    Returns:
    c: list
        List of [x, y, cell_id and None]. None will store the error "P" - Processing or "U" - Upload, if executio fails.
    grid: GeoDataFrame
        Grid file generated from input file
    adm_df: Dataframe
        Input file information
    """
    adm0_b = gpd.read_file(shp)  # adm0 base
    adm0_b = adm0_b.dissolve()
    adm0_buf = adm0_b.buffer(boundary_buffer)  # adm0 with 20KM boundary buffer
    adm0 = crs_check(adm0_buf)
    size = cell_size  # Grid cell size 0.5 ~ 55KM
    buffer = size / 2  # cell buffer around the centroid to create the cell

    grid = create_grid(adm0, size)
    # Calculate centroids and store in centroid list c[].
    c = []
    g = grid.centroid

    cell_id = 1
    for i in g:
        c.append(
            [round(i.x, 5), round(i.y, 5), cell_id, None]
        )  # The array c[] has four values: x, y, cell_id and None. None will store the "P" or "U" error value
        cell_id += 1

    n = len(c)

    if test_cells != None:
        c = c[:test_cells]
    else:
        test_cells = 0

    view_input([grid, adm0], c)

    adm_data = {
        "Parameter": [
            "Input File Path",
            "Area",
            "Area with Buffer",
            "Cell Size",
            "Total Cells",
            "Test Cells",
        ],
        "Value": [
            shp,
            "{} KM2".format(
                round((adm0_b.to_crs("EPSG:3857").area).iloc[0] / (10**6), 2)
            ),
            "{} KM2".format(
                round((adm0.to_crs("EPSG:3857").area).iloc[0] / (10**6), 2)
            ),
            "{} deg".format(cell_size),
            n,
            test_cells,
        ],
    }

    adm_df = pd.DataFrame(adm_data)
    adm_df.style.set_caption("Input Data")

    return (c, grid, adm_df)

## Outputs
## [Jump to config](#Config-Cell)

In [21]:
# Load file from sandbox disc. file should be present in 'input' folder
c, grid, adm_df = add_adm(
    shp, buffer, cell_size, test_cells
)  # (shp, boundary_buffer, cell_size)
adm_df

Visualizing data...


,Parameter,Value
0,Input File Path,input/Lake Chad.geojson
1,Area,636.94 KM2
2,Area with Buffer,636.94 KM2
3,Cell Size,0.1 deg
4,Total Cells,10
5,Test Cells,2


### Check and Run Application

In [22]:
# Init Table
count_table = gen_table()

# Calls the checklist function
if run_checks:
    e_log = exec_checks(c, cell_size / 2, count_table)
else:
    e_log = gen_aoim(c, cell_size / 2, count_table)



GRID PROCESSED AND UPLOADED SUCCESSFULLY




In [23]:
from tools.misc import get_maxmonth

get_maxmonth("output/csv/{}.csv".format(cwc_name))

In [23]:
e_log

[]

In [24]:
# Create processed and error shapefiles as grids in output/shape/
create_shapefile(c, "processed_file")
if len(e_log) > 0:
    create_shapefile(e_log, "error_file")

In [25]:
# Re-run application for cells logged in e_log, if required
if len(e_log) > 0:
    e_log = gen_aoim(e_log, cell_size)